In [19]:
import pandas as pd
import unicodedata
from difflib import SequenceMatcher

In [20]:
def normalize_name(name):
    """
    Normalize a name by:
    1. Converting to lowercase
    2. Removing accents/diacritics
    3. Stripping whitespace
    """
    if pd.isna(name):
        return ""
    # Convert to string and normalize unicode
    name = str(name)
    # Normalize unicode characters (NFD = decomposed form)
    name = unicodedata.normalize('NFD', name)
    # Remove diacritics
    name = ''.join(c for c in name if unicodedata.category(c) != 'Mn')
    # Convert to lowercase and strip
    return name.lower().strip()

In [ ]:
try:
    from rapidfuzz import fuzz, process
    USE_RAPIDFUZZ = True
except ImportError:
    USE_RAPIDFUZZ = False
    print("rapidfuzz not available, using difflib (slower but works)")

In [ ]:
def fix_encoding_issues(name):
    """
    Try to fix common encoding issues by attempting to decode/encode properly.
    Handles mojibake (UTF-8 text incorrectly decoded as Latin-1).
    """
    if pd.isna(name):
        return ""
    
    name = str(name)
    original_name = name
    
    # Common encoding fixes - try multiple strategies
    if 'Ã' in name or 'Â' in name or 'Ä' in name or 'Å' in name or 'Å¡' in name or 'Å' in name or 'Ä' in name:
        try:
            # Strategy 1: UTF-8 text that was read as Latin-1
            name = name.encode('latin-1').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            try:
                # Strategy 2: Try Windows-1252 encoding
                name = original_name.encode('windows-1252').decode('utf-8')
            except (UnicodeEncodeError, UnicodeDecodeError):
                try:
                    # Strategy 3: Try cp1252
                    name = original_name.encode('cp1252').decode('utf-8')
                except (UnicodeEncodeError, UnicodeDecodeError):
                    name = original_name
    
    return name

In [21]:
def create_name_mapping(possession_names, salary_names, threshold=85):
    """
    Create a mapping dictionary from possession names to salary names using fuzzy matching.
    
    Args:
        possession_names: list of names from player_possession_stats
        salary_names: list of names from player_salaries
        threshold: minimum similarity score (0-100) to consider a match
    
    Returns:
        dict: mapping from possession_name -> salary_name
    """
    mapping = {}
    used_salary_names = set()
    
    # First, try exact matches (case-insensitive, normalized)
    normalized_salary = {normalize_name(name): name for name in salary_names}
    
    for poss_name in possession_names:
        if pd.isna(poss_name):
            continue
            
        # Try exact match first (normalized)
        normalized_poss = normalize_name(poss_name)
        if normalized_poss in normalized_salary:
            # Find the original salary name
            for sal_name in salary_names:
                if normalize_name(sal_name) == normalized_poss:
                    mapping[poss_name] = sal_name
                    used_salary_names.add(sal_name)
                    break
            continue
        
        # If no exact match, try fuzzy matching
        if USE_RAPIDFUZZ:
            # Get the best match from salary names
            best_match, score, _ = process.extractOne(
                poss_name, 
                salary_names, 
                scorer=fuzz.WRatio
            )
            
            if score >= threshold and best_match not in used_salary_names:
                mapping[poss_name] = best_match
                used_salary_names.add(best_match)
            else:
                # Try with normalized names
                normalized_poss = normalize_name(poss_name)
                best_match_norm, score_norm, _ = process.extractOne(
                    normalized_poss,
                    [normalize_name(n) for n in salary_names],
                    scorer=fuzz.WRatio
                )
                
                if score_norm >= threshold:
                    # Find the original salary name
                    for sal_name in salary_names:
                        if normalize_name(sal_name) == best_match_norm and sal_name not in used_salary_names:
                            mapping[poss_name] = sal_name
                            used_salary_names.add(sal_name)
                            break
        else:
            # Use difflib as fallback
            best_match = None
            best_score = 0
            
            for sal_name in salary_names:
                if sal_name in used_salary_names:
                    continue
                # Calculate similarity ratio
                score = SequenceMatcher(None, poss_name.lower(), sal_name.lower()).ratio() * 100
                if score > best_score:
                    best_score = score
                    best_match = sal_name
            
            if best_score >= threshold and best_match:
                mapping[poss_name] = best_match
                used_salary_names.add(best_match)
            else:
                # Try with normalized names
                normalized_poss = normalize_name(poss_name)
                best_match_norm = None
                best_score_norm = 0
                
                for sal_name in salary_names:
                    if sal_name in used_salary_names:
                        continue
                    normalized_sal = normalize_name(sal_name)
                    score_norm = SequenceMatcher(None, normalized_poss, normalized_sal).ratio() * 100
                    if score_norm > best_score_norm:
                        best_score_norm = score_norm
                        best_match_norm = sal_name
                
                if best_score_norm >= threshold and best_match_norm:
                    mapping[poss_name] = best_match_norm
                    used_salary_names.add(best_match_norm)
    
    return mapping

def main():
    # Read the CSV files
    print("Reading player_possession_stats.csv...")
    player_possession = pd.read_csv('/Users/michaelkot/Documents/prem_2025_corr/prem_stats/player_possession_stats.csv')
    
    print("Reading player_salaries.csv...")
    player_salaries = pd.read_csv('/Users/michaelkot/Documents/prem_2025_corr/prem_stats/player_salaries.csv')
    
    # Clean up player_salaries as you mentioned
    player_salaries = player_salaries.drop(columns=['Nation', 'Position', 'Team', 'Age', 'Weekly'])
    
    # Rename 'Player' to 'player' for consistency
    player_salaries = player_salaries.rename(columns={'Player': 'player'})
    
    print(f"\nPlayer possession stats: {len(player_possession)} rows")
    print(f"Player salaries: {len(player_salaries)} rows")
    
    # Fix encoding issues in possession names first
    print("Fixing encoding issues in player names...")
    # Store original names for reference
    player_possession['player_original'] = player_possession['player'].copy()
    # Fix encoding
    player_possession['player'] = player_possession['player'].apply(fix_encoding_issues)
    
    # Get unique names from both dataframes
    possession_names = player_possession['player'].unique().tolist()
    salary_names = player_salaries['player'].unique().tolist()
    
    print(f"\nUnique players in possession stats: {len(possession_names)}")
    print(f"Unique players in salaries: {len(salary_names)}")
    
    # Create name mapping
    print("\nCreating name mapping (this may take a moment)...")
    name_mapping = create_name_mapping(possession_names, salary_names, threshold=85)
    
    print(f"\nCreated {len(name_mapping)} name mappings")
    
    # Show some examples of mappings
    print("\nSample mappings:")
    for i, (old_name, new_name) in enumerate(list(name_mapping.items())[:10]):
        if old_name != new_name:
            print(f"  '{old_name}' -> '{new_name}'")
    
    # Apply the mapping to player_possession
    print("\nApplying name mapping to player_possession...")
    player_possession['player'] = player_possession['player'].map(name_mapping).fillna(player_possession['player'])
    
    # Drop the temporary original column
    player_possession = player_possession.drop(columns=['player_original'])
    
    # Merge the dataframes
    print("\nMerging dataframes...")
    merged_df = player_possession.merge(player_salaries, on='player', how='left')
    
    # Check for NaN values in Annual column
    nan_count = merged_df['Annual'].isna().sum()
    total_count = len(merged_df)
    match_rate = (1 - nan_count / total_count) * 100
    
    print(f"\nMerge complete!")
    print(f"Total rows: {total_count}")
    print(f"Rows with salary data: {total_count - nan_count}")
    print(f"Rows without salary data (NaN): {nan_count}")
    print(f"Match rate: {match_rate:.2f}%")
    
    # Show players that didn't match
    if nan_count > 0:
        print("\nPlayers without salary matches:")
        unmatched = merged_df[merged_df['Annual'].isna()]['player'].unique()
        for player in sorted(unmatched)[:20]:  # Show first 20
            print(f"  - {player}")
        if len(unmatched) > 20:
            print(f"  ... and {len(unmatched) - 20} more")
    
    # Save the merged dataframe
    output_path = '/Users/michaelkot/Documents/prem_2025_stat_findings/csv_files_used/player_possession_with_salaries.csv'
    merged_df.to_csv(output_path, index=False)
    print(f"\nMerged data saved to: {output_path}")
    
    return merged_df, name_mapping

if __name__ == "__main__":
    merged_df, name_mapping = main()


Reading player_possession_stats.csv...
Reading player_salaries.csv...

Player possession stats: 571 rows
Player salaries: 729 rows
Fixing encoding issues in player names...

Unique players in possession stats: 559
Unique players in salaries: 729

Creating name mapping (this may take a moment)...

Created 559 name mappings

Sample mappings:

Applying name mapping to player_possession...

Merging dataframes...

Merge complete!
Total rows: 571
Rows with salary data: 571
Rows without salary data (NaN): 0
Match rate: 100.00%

Merged data saved to: /Users/michaelkot/Documents/prem_2025_stat_findings/csv_files_used/player_possession_with_salaries.csv
